### 형태소 분리 먼저 하지 말고, 원문 그대로 가져와 감성 분석

In [1]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import json
from konlpy.tag import Okt
from konlpy.corpus import kolaw
import nltk

**제목 그대로 가져오기**

In [55]:
data = pd.read_csv('./data/all_news_data.csv', sep = ",",  encoding='cp949')
title = data['제목']
print(title)
print(len(title))
print(type(title))

0            시원스쿨 평생수강권 무료, 카카오페이지 OOOO 들어갈 정답 '관심'
1                       에스원 - 카카오모빌리티, 무인 파킹 솔루션 출시
2                   카카오 ‘性착취 금지·아동 性 보호’ 모든 서비스에 적용
3        카카오, '성범죄 행위 금지' 명문화… 'N번방법' 시행 앞두고 선제적 대응
4                             '테크 스타트업'에 꽂힌 네이버·카카오
                            ...                    
14885               인간중심 기술 위한 한국 기업의 역할[기고/필립 코틀러]
14886         신용 하위50% 늘어난다...인터넷銀 중·저신용 대출확대 '청신호'
14887            "바이오가 되겠냐 했는데 이젠 금맥"…롯데·GS·신세계도 참전
14888                "대출 받아가세요"…굳게 닫혔던 은행 문이 다시 열렸다
14889             [단신]신한은행, 우편 안내문 대신 모바일 전자문서 발송 外
Name: 제목, Length: 14890, dtype: object
14890
<class 'pandas.core.series.Series'>


**제목 줄별로 형태소 분석해서 다시 리스트에 각각 집어넣기**

In [3]:
okt = Okt()

In [4]:
okt_list = []

for i in range(len(title)):
    okt_list.append(okt.morphs(title[i]))

print(okt_list)
print(len(okt_list))

[['시원', '스쿨', '평생', '수강', '권', '무료', ',', '카카오페이지', 'OOOO', '들어갈', '정답', "'", '관심', "'"], ['에스원', '-', '카카오', '모빌리티', ',', '무인', '파킹', '솔루션', '출시'], ['카카오', '‘性', '착취', '금지', '·', '아동', '性', '보호', '’', '모든', '서비스', '에', '적용'], ['카카오', ',', "'", '성범죄', '행위', '금지', "'", '명문', '화', '…', "'", 'N', '번', '방법', "'", '시행', '앞두고', '선제', '적', '대응'], ["'", '테크', '스타트업', "'", '에', '꽂힌', '네이버', '·', '카카오'], ['檢', ',', '한동훈', '유심', '칩', '으로', '카카오', '톡', '들여다본', '듯', '...', '불법', '감청', '논란', '도'], ['네이버', '·', '카카오', '페이', ',', '30만원', '까지', '후불', '결제', '허용'], ['네이버', 'vs', '카카오', 'vs', '토스', '...', '빅', '테크', '각자', '도생', '시작', '됐다'], ["'", '3000억', '혈맹', "'", 'SKT', '·', '카카오', '"', '협업', '쉽지', '않네', '"'], ['"', '네이버', '·', '카카오', '는', '세상', '바꿀', '주식', '…', 'PER', '등', '기존', '잣', '대로', '가치', '평가', '금물', '"'], ['카카오', '주식', '액면', '분할', '한', '다', '…', '500원', '→', '100원'], ['시동', '끈', '타다', '드라이버', '4', '명', '중', '1', '명', "'", '카카오', "'", '로'], ['성과급', '문제', '소통', '나선', '네이버', '와', '카카오', '…', '네이버

**형태소 분리된 각 줄별로 긍정/부정/중립 분석**

In [46]:
# import nltk
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
nltk.pos_tag(okt_list[1])

[('에스원', 'SYM'),
 ('-', ':'),
 ('카카오', 'NN'),
 ('모빌리티', 'NN'),
 (',', ','),
 ('무인', 'NNP'),
 ('파킹', 'NNP'),
 ('솔루션', 'NNP'),
 ('출시', 'NN')]

In [57]:
pos_list = []
num = []

for i in range(len(okt_list)):
    num.append(i)
    pos_list.append(nltk.pos_tag(okt_list[i]))

title_df = pd.DataFrame({'번호':num, '토큰화':pos_list})
title_df

,번호,토큰화
0,0,"[(시원, JJ), (스쿨, NNP), (평생, NNP), (수강, NNP), (권..."
1,1,"[(에스원, SYM), (-, :), (카카오, NN), (모빌리티, NN), (,..."
2,2,"[(카카오, JJ), (‘性, NNP), (착취, NNP), (금지, NNP), (..."
3,3,"[(카카오, NN), (,, ,), (', ''), (성범죄, JJ), (행위, N..."
4,4,"[(', POS), (테크, JJ), (스타트업, NNP), (', POS), (에..."
...,...,...
14885,14885,"[(인간, JJ), (중심, NNP), (기술, NNP), (위, NNP), (한,..."
14886,14886,"[(신용, JJ), (하위, $), (50%, CD), (늘어난다, NN), (....."
14887,14887,"[("", JJ), (바이오, NNP), (가, NNP), (되겠냐, NNP), (했..."
14888,14888,"[("", JJ), (대출, NNP), (받아가세요, NNP), (""…, NNP), ..."


In [58]:
title_df['원문'] = title

In [69]:
news_data = pd.read_csv('./data/news_all_data.csv', encoding= 'cp949')
news_data

,Unnamed: 0,...1,제목,날짜
0,1,16,"시원스쿨 평생수강권 무료, 카카오페이지 OOOO 들어갈 정답 '관심'",0
1,2,91,"에스원 - 카카오모빌리티, 무인 파킹 솔루션 출시",0
2,3,81,카카오 ‘性착취 금지·아동 性 보호’ 모든 서비스에 적용,0
3,4,97,"카카오, '성범죄 행위 금지' 명문화… 'N번방법' 시행 앞두고 선제적 대응",0
4,5,61,'테크 스타트업'에 꽂힌 네이버·카카오,0
...,...,...,...,...
14885,14886,199,인간중심 기술 위한 한국 기업의 역할[기고/필립 코틀러],2022.04.11.
14886,14887,221,신용 하위50% 늘어난다...인터넷銀 중·저신용 대출확대 '청신호',2022.04.11.
14887,14888,229,"""바이오가 되겠냐 했는데 이젠 금맥""…롯데·GS·신세계도 참전",2022.04.11.
14888,14889,243,"""대출 받아가세요""…굳게 닫혔던 은행 문이 다시 열렸다",2022.04.11.


In [71]:
# 날짜, 원문, 토큰화 모두 한 df에 담긴
title_df = title_df[['원문','토큰화']]
title_df['날짜'] = news_data['날짜']
title_df = title_df[['날짜','원문','토큰화']]
title_df

,날짜,원문,토큰화
0,0,"시원스쿨 평생수강권 무료, 카카오페이지 OOOO 들어갈 정답 '관심'","[(시원, JJ), (스쿨, NNP), (평생, NNP), (수강, NNP), (권..."
1,0,"에스원 - 카카오모빌리티, 무인 파킹 솔루션 출시","[(에스원, SYM), (-, :), (카카오, NN), (모빌리티, NN), (,..."
2,0,카카오 ‘性착취 금지·아동 性 보호’ 모든 서비스에 적용,"[(카카오, JJ), (‘性, NNP), (착취, NNP), (금지, NNP), (..."
3,0,"카카오, '성범죄 행위 금지' 명문화… 'N번방법' 시행 앞두고 선제적 대응","[(카카오, NN), (,, ,), (', ''), (성범죄, JJ), (행위, N..."
4,0,'테크 스타트업'에 꽂힌 네이버·카카오,"[(', POS), (테크, JJ), (스타트업, NNP), (', POS), (에..."
...,...,...,...
14885,2022.04.11.,인간중심 기술 위한 한국 기업의 역할[기고/필립 코틀러],"[(인간, JJ), (중심, NNP), (기술, NNP), (위, NNP), (한,..."
14886,2022.04.11.,신용 하위50% 늘어난다...인터넷銀 중·저신용 대출확대 '청신호',"[(신용, JJ), (하위, $), (50%, CD), (늘어난다, NN), (....."
14887,2022.04.11.,"""바이오가 되겠냐 했는데 이젠 금맥""…롯데·GS·신세계도 참전","[("", JJ), (바이오, NNP), (가, NNP), (되겠냐, NNP), (했..."
14888,2022.04.11.,"""대출 받아가세요""…굳게 닫혔던 은행 문이 다시 열렸다","[("", JJ), (대출, NNP), (받아가세요, NNP), (""…, NNP), ..."


In [72]:
title_df.to_csv('./data/news_token.csv')

### 형태소별로 긍정/부정/중립 나누기

In [29]:
title = pd.read_csv('./data/news_token.csv', encoding= 'cp949')

In [23]:
# 부정단어 file 리스트화
f = open('./data/negative_words_self.txt', encoding='utf-8')
f
negative = f.readlines()
neg = [neg.replace('\n','') for neg in negative]
neg

['갑질',
 '논란',
 '폭리',
 '허위',
 '과징금',
 '눈물',
 '피해',
 '포화',
 '우롱',
 '위반',
 '리스크',
 '사퇴',
 '급락',
 '하락',
 '폐업',
 '불만',
 '산재',
 '닫아',
 '손배소',
 '구설수',
 '폐점',
 '적발',
 '침해',
 '빨간불',
 '취약',
 '불명예',
 '분신',
 '구형',
 '기소',
 '반토막',
 '호소',
 '불매',
 '냉담',
 '문제',
 '직격탄',
 '한숨',
 '불똥',
 '항의',
 '묵묵부담',
 '싸늘',
 '일탈',
 '파문',
 '게이트',
 '횡령',
 '사과문',
 '여파',
 '울상',
 '초토화',
 '급감',
 '우려',
 '중단',
 '퇴출',
 '해지',
 '일베',
 '이물질',
 '벌레',
 '엉망',
 '적발',
 '위생불량',
 '위생엉망',
 '소송',
 '하락',
 '매출하락',
 '반토막',
 '혐의',
 '불매',
 '부채',
 '적발',
 '폭리',
 '과징금',
 '오너리스크',
 '매각',
 '포기',
 '불만']

In [24]:
# 긍정단어 file 리스트화
f = open('./data/positive_words_self.txt', encoding='utf-8')
positive = f.readlines()
pos = [pos.replace('\n','') for pos in positive]
pos

['MOU',
 '제휴',
 '주목',
 '호응',
 '돌파',
 '이목',
 '수상',
 '입점',
 '인기',
 '열풍',
 '진화',
 '대박',
 '순항',
 '유치',
 '1위',
 '출시',
 '선보여',
 '오픈',
 '팝업',
 '돌풍',
 '팝업스토어',
 '인싸',
 '줄서서',
 '인기',
 '대세',
 '트렌드',
 '불티',
 '진출',
 '부상',
 '체결',
 '증가',
 '봉사',
 '기부',
 '신메뉴',
 '신제품',
 '신상',
 '최고',
 '새로운',
 '편한',
 '미소',
 '맛집',
 '착한가게',
 '캠패인',
 '순항',
 '착한',
 '제작지원']

In [54]:
if title_df[0][0][0] != pos[0]:
    print(False)

False
